## Upload the data on your instance

To train a model using Sagemaker, you first need to upload your data on S3.

- zip your data and upload it on the instance
- extract the data from data.zip and remove data.zip


In [ ]:
!unzip data.zip
!rm data.zip

## Upload the data on S3

In [2]:
import sagemaker

In [34]:
bucket = sagemaker.Session().default_bucket()

prefix = 'founder-classifier/data'

train_response = sagemaker.Session().upload_data(path='data/train.json', 
                                         bucket=bucket, 
                                         key_prefix=prefix)

test_response = sagemaker.Session().upload_data(path='data/test.json', 
                                         bucket=bucket, 
                                         key_prefix=prefix)

print(train_response)
print(test_response)

s3://sagemaker-eu-west-1-107382316743/founder-classifier/data/train.json
s3://sagemaker-eu-west-1-107382316743/founder-classifier/data/test.json


## Entry Script

In [12]:
!cat aws_entry_point.py

## Launch Training

In [ ]:
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

role=get_execution_role()

estimator = TensorFlow(entry_point='aws_entry_point.py',
                             role=role,
                             training_steps=1000, 
                             train_instance_count=1,
                             train_instance_type='ml.p2.xlarge',
                             hyperparameters={
                                              'learning_rate': 0.001,
                                              'batch_size' : 32
                                      })

In [ ]:
estimator.fit('s3://sagemaker-eu-west-1-107382316743/founder-classifier/data')

## Deploy your model

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, 
                             instance_type='ml.p2.xlarge',
                             endpoint_name='founder-classifier-endpoint')

In [ ]:
import cv2
import numpy as np

#lets test if the endpoint works by throwing it some image
location='./data/bing/Jeff Bezos/faces_28x28/Jeff_Bezos_100.jpg'
image=cv2.imread(location, cv2.IMREAD_GRAYSCALE)
image=np.array(image, dtype='float64').reshape([-1,28,28,1])

In [ ]:
predictor.predict({'x': image})

In [45]:
sagemaker.Session().delete_endpoint(predictor.endpoint)